## 1. Import packages

In [1]:
#!pip install tweepy

In [1]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm
import gc #garbage collector
gc.collect()

0

## 2. Twitter API authentication

In [2]:
consumer_api_key = 'Type you API KEY here '
consumer_api_secret = 'Type your API KEY SECRET here'

In [3]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [4]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [38]:
# If you want to extract tweets with the hashtag #covidvaccine! then enter the hastag in the search_words variable
search_words = "#CovidVaccine -filter:retweets"

#For paticular time frames you could assign variables as per your need!
date_since = "2020-07-18"
date_until="2020-08-1"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since,until=date_until).items(7500)

### 3.2. Retreive the tweets

In [39]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)


0it [00:00, ?it/s]


In [37]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 2636


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))


 60%|███████████████████████████████████████████████                               | 1634/2709 [00:14<00:11, 95.64it/s]


100%|█████████████████████████████████████████████████████████████████████████████| 2709/2709 [00:26<00:00, 103.71it/s]


In [11]:
tweets_df.to_csv('covid_vaccine.csv',index=False)